Eodag basics
===========

We present the two basic features of eodag : search and download.

Let's start by creating a workspace directory where all our files and configuration will live:

In [1]:
import os

workspace = 'eodag_workspace'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

Set your personal credentials to access [peps service](https://peps.cnes.fr):

In [1]:
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "PLEASE_CHANGE_ME"
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "PLEASE_CHANGE_ME"

Next, we consider that we have a configuration file sitting in our workspace called `eodag_conf.yml`, with this minimalist content:

    peps:
        download:
            outputs_prefix: <absolute_path_to_eodag_workspace>
            extract: true
    
The rest of the configuration is the providers credentials given as described in the documentation.

The first step is to initialize the session by creating an `EODataAccessGateway` instance, with the previous configuration file:

In [2]:
# To have some basic feedback on what eodag is doing, we configure logging to output minimum information
from eodag.utils.logging import setup_logging
setup_logging(verbose=1)

from eodag.api.core import EODataAccessGateway

dag = EODataAccessGateway()

2019-04-15 16:30:09,186-15s eodag.config                     [INFO    ] Loading user configuration from: /tmp/eodag_workspace/eodag_conf.yml


Now let's search for Sentinel 2 L1C products in the South of France:

In [3]:
from datetime import date

product_type = 'S2_MSI_L1C'
extent = {
    'lonmin': -1.999512,
    'lonmax': 4.570313,
    'latmin': 42.763146,
    'latmax': 46.754917
}

products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start='2018-06-01',
    end=date.today().isoformat(),
    box=extent
)

2019-04-15 16:36:12,426-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2019-04-15 16:36:12,428-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?box=-1.999512,42.763146,4.570313,46.754917&startDate=2018-06-01&productType=S2MSI1C&completionDate=2019-04-15&maxRecords=1&page=1
2019-04-15 16:36:14,423-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?box=-1.999512,42.763146,4.570313,46.754917&startDate=2018-06-01&productType=S2MSI1C&completionDate=2019-04-15&maxRecords=20&page=1
2019-04-15 16:36:17,106-15s eodag.core                       [INFO    ] Found 2090 result(s) on provider 'peps'


Now that we have found some products, we can store the search result in a geojson file that will be used for later download:

In [4]:
search_result_file = dag.serialize(products, filename=os.path.join(workspace, 'search_results.geojson'))

We can check the extents of the products before downloading them by using ipyleaflet API:

In [5]:
import ipyleaflet as ipyl

m = ipyl.Map(center=[43.6, 1.5], zoom=5)
layer = ipyl.GeoJSON(data=products.as_geojson_object(), hover_style={'fillColor': 'yellow'})

m.add_layer(layer)
m

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


Finally, we can download from the eodag SearchResult object `products`, (or from the GeoJson previously created):

In [6]:
from eodag.utils import ProgressCallback
from tqdm import tqdm_notebook

# Create a custom progressbar callback
class NotebookProgressCallback(ProgressCallback):
    
    def __call__(self, current_size, max_size):
        if self.pb is None:
            self.pb = tqdm_notebook(total=max_size, unit='KB', unit_scale=True)
        self.pb.update(current_size)


# Download the first product found
path = dag.download(products[0], progress_callback=NotebookProgressCallback())

2019-04-15 16:37:01,419-15s eodag.plugins.download.http      [INFO    ] Download url: https://peps.cnes.fr/resto/collections/S2ST/1c43894b-1248-5768-af70-e38a59ca8601/download


2019-04-15 16:41:34,195-15s eodag.plugins.download.http      [INFO    ] Extraction activated
Extracting files from S2B_MSIL1C_20190414T105629_N0207_R094_T30TXP_20190414T120932.zip: 100%|██████████| 115/115 [00:58<00:00,  1.96file/s]
2019-04-15 16:42:32,964-15s eodag.api.product                [INFO    ] Remote location of the product is still available through its 'remote_location' property: https://peps.cnes.fr/resto/collections/S2ST/1c43894b-1248-5768-af70-e38a59ca8601/download


This concludes the tutorial for the basic usage of eodag.